In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import torch
from torchvision import transforms
from torch.utils.data.dataset import Dataset
from neuralop.models.fno import TFNO
from neuralop.models.fno import TFNO3d
from neuralop.training.trainer import Trainer
from neuralop.utils import count_model_params
from neuralop.losses.data_losses import LpLoss, H1Loss
import pdb
import torch.nn as nn
import sys
from astropy.io import fits
from torch.utils.data.dataset import Subset
import random
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Subset
from torchvision.transforms.functional import normalize
from matplotlib.animation import FuncAnimation, FFMpegWriter
from IPython.display import HTML
from tqdm import tqdm
from torch.utils.data import random_split
import h5py
import torch.nn as nn
import time

device0 = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#device1 = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

print(device0)
#print(device1)

from functools import partialmethod
import torch
import torch.nn as nn
import torch.nn.functional as F
from neuralop.layers.spectral_convolution import SpectralConv
from neuralop.layers.padding import DomainPadding
from neuralop.layers.fno_block import FNOBlocks
from neuralop.layers.mlp import MLP
from neuralop.models.base_model import BaseModel

In [ ]:
import torch
from torch.utils.data import Dataset
import numpy as np
import h5py
import pandas as pd

class CustomDataset(Dataset):
    def __init__(
        self, 
        hdf5_file_paths, 
        x_lengths, 
        y_lengths, 
        z_lengths, 
        excel_path
    ):
        """
        Args:
            hdf5_file_paths (list): List of paths to the HDF5 files [Bx_file, By_file, Bz_file].
            x_lengths, y_lengths, z_lengths (np.ndarray): Arrays of lengths (1D).
            excel_path (str): Path to the Excel file containing NOAA AR, date, and type.
        """
        # 1) Store HDF5 file paths and open the files
        self.hdf5_file_paths = hdf5_file_paths
        self.hdf5_files = [h5py.File(path, 'r') for path in self.hdf5_file_paths]
        
        # 2) Store lengths
        self.x_lengths = x_lengths
        self.y_lengths = y_lengths
        self.z_lengths = z_lengths
        
        # 3) Read Excel file to get the NOAA AR, date, and type
        df = pd.read_excel(excel_path)
        
        # Make sure the rows in 'df' match the exact order of samples in HDF5
        # If you need to sort the DataFrame, do it here, e.g.:
        # df = df.sort_values(by='date')  # or by 'NOAA AR', etc.
        # df = df.reset_index(drop=True)

        # Just store the raw type strings:
        self.type_labels_str = df['type'].astype(str).values
        
        # 4) Assume all HDF5 files have the same number of samples
        self.dataset_length = len(self.hdf5_files[0])
        
        # 5) Calculate means and stds for normalization
        self.x_mean = np.mean(self.x_lengths)
        self.x_std = np.std(self.x_lengths)

        self.y_mean = np.mean(self.y_lengths)
        self.y_std = np.std(self.y_lengths)

        self.z_mean = np.mean(self.z_lengths)
        self.z_std = np.std(self.z_lengths)

    def __len__(self):
        return self.dataset_length

    def __getitem__(self, idx):
        # ---------------------
        # 1) Read input arrays
        # ---------------------
        input_data_x = torch.from_numpy(
            self.hdf5_files[0][f'sample_{idx}/input'][:]
        ).float()
        input_data_y = torch.from_numpy(
            self.hdf5_files[1][f'sample_{idx}/input'][:]
        ).float()
        input_data_z = torch.from_numpy(
            self.hdf5_files[2][f'sample_{idx}/input'][:]
        ).float()

        # ----------------------
        # 2) Read output arrays
        # ----------------------
        output_data_x = torch.from_numpy(
            self.hdf5_files[0][f'sample_{idx}/output'][:]
        ).float()
        output_data_y = torch.from_numpy(
            self.hdf5_files[1][f'sample_{idx}/output'][:]
        ).float()
        output_data_z = torch.from_numpy(
            self.hdf5_files[2][f'sample_{idx}/output'][:]
        ).float()

        # ------------------------------------
        # 3) Select some subset of output data
        # ------------------------------------
        selected_output_data_x = output_data_x[:25]
        selected_output_data_y = output_data_y[:25]
        selected_output_data_z = output_data_z[:25]

        # ------------------------------------
        # 4) Stack inputs into multi-channel
        # ------------------------------------
        input_data = torch.stack(
            (input_data_x, input_data_y, input_data_z), dim=0
        ).unsqueeze(0)

        # ------------------------------------
        # 5) Stack selected outputs
        # ------------------------------------
        output_data = torch.stack(
            (selected_output_data_x, selected_output_data_y, selected_output_data_z), 
            dim=1
        )

        # ------------------------------------
        # 6) Normalize lengths
        # ------------------------------------
        x_length_normalized = (self.x_lengths[idx] - self.x_mean) / self.x_std
        y_length_normalized = (self.y_lengths[idx] - self.y_mean) / self.y_std
        z_length_normalized = (self.z_lengths[idx] - self.z_mean) / self.z_std
        
        length_tensor = torch.tensor(
            [x_length_normalized, y_length_normalized, z_length_normalized],
            dtype=torch.float32
        ).unsqueeze(0)

        # ------------------------------------
        # 7) Get the string label from Excel
        # ------------------------------------
        type_label_str = self.type_labels_str[idx]
        # This remains a Python string. If you only need to read it
        # later for grouping or analysis, you can just store it in the sample dict. 
        # (If you need it as a tensor, you can store it as an object 
        #  but note that PyTorch won't do much with it for training.)

        sample = {
            'input': input_data,      # shape: (1, 3, H, W) or similar
            'output': output_data,    # shape: (N, 3) depending on your slicing
            'lengths': length_tensor, # shape: (1, 3)
            'type_label_str': type_label_str,  # e.g. 'β' or 'βγ'
        }
        return sample


from torch.utils.data import DataLoader, Subset

# HDF5 file paths
input1 = "NLFFF_data/NLFFF_Bx_rmns_100.h5"
input2 = "NLFFF_data/NLFFF_By_rmns_100.h5"
input3 = "NLFFF_data/NLFFF_Bz_rmns_10.h5"

# NumPy length arrays
x_lengths = np.load("NLFFF_data/x_height.npy")
y_lengths = np.load("NLFFF_data/y_height.npy")
z_lengths = np.load("NLFFF_data/z_height.npy")

# Excel path
excel_path = "NLFFF_data/NLFFF_datatype.xlsx"

# Initialize dataset
dataset = CustomDataset(
    [input1, input2, input3],
    x_lengths,
    y_lengths,
    z_lengths,
    excel_path
)

train_size = 143
test_size = 27

train_dataset = Subset(dataset, range(train_size))
test_dataset = Subset(dataset, range(train_size, train_size + test_size))

train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True, num_workers=8, persistent_workers=False)
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False, num_workers=8, persistent_workers=False)

for batch in train_loader:
    inputs = batch['input']
    outputs = batch['output']
    lengths = batch['lengths']
    type_strings = batch['type_label_str']

print(inputs.shape)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class PhysicsInformedLoss(nn.Module):
    def __init__(self, weight=0.35, alpha=1.0, beta=1.0, gamma=1.0):
        super(PhysicsInformedLoss, self).__init__()
        self.weight = weight
        self.alpha = alpha  
        self.beta = beta    
        self.gamma = gamma  

    def forward(self, output, target): 
        divergence_loss = self.compute_divergence_loss(output)

        force_free_loss = self.compute_force_free_loss(output)

        total_loss = self.weight * ((self.beta * divergence_loss) + \
                     (self.gamma * force_free_loss))
        return total_loss

    def compute_divergence_loss(self, output):
        batch_size, depth, components, height, width = output.shape
        B = output.permute(0, 2, 1, 3, 4)  # B: [batch_size, 3, depth, height, width]
        channels = components

        device = output.device

        kernel_dx = torch.tensor([-0.5, 0, 0.5], device=device).reshape(1, 1, 1, 1, 3)
        kernel_dy = torch.tensor([-0.5, 0, 0.5], device=device).reshape(1, 1, 1, 3, 1)
        kernel_dz = torch.tensor([-0.5, 0, 0.5], device=device).reshape(1, 1, 3, 1, 1)
        kernel_dx = kernel_dx.repeat(channels, 1, 1, 1, 1)
        kernel_dy = kernel_dy.repeat(channels, 1, 1, 1, 1)
        kernel_dz = kernel_dz.repeat(channels, 1, 1, 1, 1)

        dB_dx = F.conv3d(B, kernel_dx, padding=(0, 0, 1), groups=channels)
        dB_dy = F.conv3d(B, kernel_dy, padding=(0, 1, 0), groups=channels)
        dB_dz = F.conv3d(B, kernel_dz, padding=(1, 0, 0), groups=channels)

        # dB_x/dx
        dB_x_dx = dB_dx[:, 0, :, :, :]
        # dB_y/dy
        dB_y_dy = dB_dy[:, 1, :, :, :]
        # dB_z/dz
        dB_z_dz = dB_dz[:, 2, :, :, :]


        divergence = dB_x_dx + dB_y_dy + dB_z_dz
        divergence_loss = torch.mean(divergence ** 2)

        return divergence_loss

    def compute_force_free_loss(self, output):
        batch_size, depth, components, height, width = output.shape
        B = output.permute(0, 2, 1, 3, 4)  # B: [batch_size, 3, depth, height, width]
        channels = components  

        device = output.device

        kernel_dx = torch.tensor([-0.5, 0, 0.5], device=device).reshape(1, 1, 1, 1, 3)
        kernel_dy = torch.tensor([-0.5, 0, 0.5], device=device).reshape(1, 1, 1, 3, 1)
        kernel_dz = torch.tensor([-0.5, 0, 0.5], device=device).reshape(1, 1, 3, 1, 1)

        kernel_dx = kernel_dx.repeat(channels, 1, 1, 1, 1)
        kernel_dy = kernel_dy.repeat(channels, 1, 1, 1, 1)
        kernel_dz = kernel_dz.repeat(channels, 1, 1, 1, 1)

        dB_dx = F.conv3d(B, kernel_dx, padding=(0, 0, 1), groups=channels)
        dB_dy = F.conv3d(B, kernel_dy, padding=(0, 1, 0), groups=channels)
        dB_dz = F.conv3d(B, kernel_dz, padding=(1, 0, 0), groups=channels)

        dB_x_dy = dB_dy[:, 0, :, :, :]
        dB_x_dz = dB_dz[:, 0, :, :, :]

        dB_y_dx = dB_dx[:, 1, :, :, :]
        dB_y_dz = dB_dz[:, 1, :, :, :]

        dB_z_dx = dB_dx[:, 2, :, :, :]
        dB_z_dy = dB_dy[:, 2, :, :, :]

        rot_x = dB_z_dy - dB_y_dz
        rot_y = dB_x_dz - dB_z_dx
        rot_z = dB_y_dx - dB_x_dy

        j = torch.stack([rot_x, rot_y, rot_z], dim=1)
        jxb = torch.cross(j, B, dim=1)
        B_magnitude_squared = torch.sum(B ** 2, dim=1, keepdim=True) + 1e-7
        force_free_loss = torch.mean(torch.sum(jxb ** 2, dim=1) / B_magnitude_squared.squeeze(1))

        return force_free_loss


class ECA3DLayer(nn.Module):
    def __init__(self, channel, k_size=3):
        super(ECA3DLayer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool3d(1)
        self.conv = nn.Conv1d(1, 1, kernel_size=k_size, padding=(k_size - 1) // 2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # x: [batch_size, channels, seq_len, height, width]
        
        # Global average pooling across seq_len, height, width
        y = self.avg_pool(x)  # Shape: [batch_size, channels, 1, 1, 1]

        # Apply 1D convolution across the channel dimension
        y = self.conv(y.squeeze(-1).squeeze(-1).transpose(-1, -2))  # Shape: [batch_size, 1, channels]
        y = y.transpose(-1, -2).unsqueeze(-1).unsqueeze(-1)  # Shape: [batch_size, channels, 1, 1, 1]

        # Apply attention to input
        return x * self.sigmoid(y).expand_as(x)

class DilatedConvYWithECA(nn.Module):
    def __init__(self, channels, dilation_y=2, k_size=3):
        super(DilatedConvYWithECA, self).__init__()
        self.conv = nn.Conv3d(
            in_channels=channels,
            out_channels=channels,
            kernel_size=(1, 3, 1),  
            padding=(0, dilation_y, 0),  
            dilation=(1, dilation_y, 1),
            bias=False
        )
        self.activation = nn.ReLU()

        # ECA Attention layer
        self.eca = ECA3DLayer(channel=channels, k_size=k_size)

    def forward(self, x):
        # Apply dilated convolution
        x = self.conv(x)
        x = self.activation(x)

        # Apply ECA attention
        x = self.eca(x)
        
        return x

# CustomFNO Model with ECA-Net applied to lengths_lifted
class CustomFNO(BaseModel, name='CustomFNO'):
    def __init__(self, n_modes, hidden_channels, in_channels=3, out_channels=1,
                 lifting_channels=256, projection_channels=256, n_layers=4,
                 output_scaling_factor=None, max_n_modes=None,
                 fno_block_precision="full", use_mlp=False, mlp_dropout=0,
                 mlp_expansion=0.5, non_linearity=F.gelu, stabilizer=None,
                 norm=None, preactivation=False, fno_skip="linear",
                 mlp_skip="soft-gating", separable=False, factorization=None,
                 rank=1.0, joint_factorization=False, fixed_rank_modes=False,
                 implementation="factorized", decomposition_kwargs=dict(),
                 domain_padding=None, domain_padding_mode="one-sided",
                 fft_norm="forward", SpectralConv=SpectralConv,
                 **kwargs):
        
        super().__init__()
        self.n_dim = len(n_modes)
        self.n_layers = n_layers

        # Define the lifting layers for two inputs
        self.lifting1 = MLP(
            in_channels=in_channels,
            out_channels=hidden_channels,
            hidden_channels=lifting_channels,
            n_layers=2,
            n_dim=self.n_dim,
        )
        
        self.lifting_y = MLP(
            in_channels=3,  
            out_channels=hidden_channels,
            hidden_channels=lifting_channels,
            n_layers=2,
            n_dim=1,
        )

        # Define the ECA-Net block for lengths_lifted
        self.DilatedConvYWithECA = DilatedConvYWithECA(hidden_channels, dilation_y=2)

        # Define the FNO blocks
        self.fno_blocks = FNOBlocks(
            in_channels=hidden_channels,
            out_channels=hidden_channels,
            n_modes=n_modes,
            output_scaling_factor=output_scaling_factor,
            use_mlp=use_mlp,
            mlp_dropout=mlp_dropout,
            mlp_expansion=mlp_expansion,
            non_linearity=non_linearity,
            stabilizer=stabilizer,
            norm=norm,
            preactivation=preactivation,
            fno_skip=fno_skip,
            mlp_skip=mlp_skip,
            max_n_modes=max_n_modes,
            fno_block_precision=fno_block_precision,
            rank=rank,
            fft_norm=fft_norm,
            fixed_rank_modes=fixed_rank_modes,
            implementation=implementation,
            separable=separable,
            factorization=factorization,
            decomposition_kwargs=decomposition_kwargs,
            joint_factorization=joint_factorization,
            SpectralConv=SpectralConv,
            n_layers=n_layers,
            **kwargs
        )

        # Define the projection layer
        self.projection = MLP(
            in_channels=hidden_channels,
            out_channels=out_channels,
            hidden_channels=projection_channels,
            n_layers=2,
            n_dim=self.n_dim,
            non_linearity=non_linearity,
        )

    def forward(self, x1, lengths):
        """Forward pass for the Custom FNO model.

        Parameters
        ----------
        x1 : tensor
            Input tensor of shape [batch_size, 1, 3, 257, 513]
        lengths : tensor
            Tensor containing the normalized lengths of shape [batch_size, 1, 3]
        """
        x_orig = x1.clone()
        
        # Apply lifting to the input
        x1 = self.lifting1(x1)  # Result shape: [batch_size, hidden_channels, 3, 257, 513]

        # Reshape lengths for processing
        lengths = lengths.view(-1, 3, 1)  # Shape [batch_size, 3, 1] for processing

        # Process lengths through lifting
        lengths_lifted = self.lifting_y(lengths)  # Result shape: [batch_size, hidden_channels, 3]
        
        # Reshape lengths_lifted to match the spatial dimensions of x1
        lengths_lifted = lengths_lifted.unsqueeze(-1).unsqueeze(-1)  # Shape: [batch_size, hidden_channels, 3, 1, 1]

        # Apply ECA-Net block to lengths_lifted before adding to x1
        lengths_lifted = self.DilatedConvYWithECA(lengths_lifted)

        # Apply lengths_lifted to x1 (add)
        x1 = x1 + lengths_lifted

        # Apply FNO blocks
        for layer_idx in range(self.n_layers):
            x1 = self.fno_blocks(x1, layer_idx)

        # Apply projection layer
        x1 = self.projection(x1)

        x1[:, 0, :, :, :] = x_orig[:, 0, :, :, :]

        return x1

In [ ]:
model = CustomFNO(n_modes = (32, 32, 32), in_channels=1, out_channels=25, n_layers = 6,
               hidden_channels=32, use_mlp=True, factorization='tucker', rank=0.2)
model = model.to(device0)

n_params = count_model_params(model)
print(f'\nYour model has {n_params} parameters.')
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4) 
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.8, patience=1, verbose=True)

h1loss = H1Loss(d=2)
print(model)

In [ ]:
def l2_loss(pred, target):
    return F.mse_loss(pred, target, reduction='mean')

In [ ]:
import os
import time
import csv
import torch.nn.functional as F
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from skimage.metrics import peak_signal_noise_ratio as psnr, structural_similarity as ssim
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

num_epochs = 150
print_frequency = 40
save_checkpoint_start_epoch = 50
save_checkpoint_interval = 10
checkpoint_dir = "result/NLFFF_Cube2Step/step1"
results_file = os.path.join(checkpoint_dir, "result.csv")
physics_loss_fn = PhysicsInformedLoss(weight=0.35, alpha=1.0, beta=1, gamma=1)

# 准备CSV文件
with open(results_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['epoch', 'comp', 'r2', 'relative_error', 'mse', 'mae', 'psnr', 'ssim'])

train_losses_Bx = []
train_losses_By = []
train_losses_Bz = []
test_losses_Bx = []
test_losses_By = []
test_losses_Bz = []
physics_losses = []
epoch_durations = []

for epoch in range(num_epochs):
    start_time = time.time()

    model.train()
    epoch_loss_Bx = 0.0
    epoch_loss_By = 0.0
    epoch_loss_Bz = 0.0
    epoch_physics_loss = 0.0

    train_loader_tqdm = tqdm(enumerate(train_loader), total=len(train_loader), desc=f'Train Epoch {epoch+1}/{num_epochs}')
    for batch_idx, sample in train_loader_tqdm:
        optimizer.zero_grad()
        data, target = sample['input'].to(device0), sample['output'].to(device0)
        lengths = sample['lengths'].to(device0)  

        output = model(data, lengths)

        loss_Bx = h1loss(output[:, :, 0, :, :], target[:, :, 0, :, :]).mean()
        loss_By = h1loss(output[:, :, 1, :, :], target[:, :, 1, :, :]).mean()
        loss_Bz = h1loss(output[:, :, 2, :, :], target[:, :, 2, :, :]).mean()

        physics_loss = physics_loss_fn(output, target)
            
        total_loss = loss_Bx + loss_By + loss_Bz + physics_loss
        total_loss.backward()
        optimizer.step()

        epoch_loss_Bx += loss_Bx.item()
        epoch_loss_By += loss_By.item()
        epoch_loss_Bz += loss_Bz.item()
        epoch_physics_loss += physics_loss.item()

        if batch_idx % print_frequency == 0:
            train_loader_tqdm.set_postfix(
                loss_Bx=loss_Bx.item(),
                loss_By=loss_By.item(),
                loss_Bz=loss_Bz.item(),
                physics_loss=physics_loss.item()
            )

    avg_loss_Bx = epoch_loss_Bx / len(train_loader)
    avg_loss_By = epoch_loss_By / len(train_loader)
    avg_loss_Bz = epoch_loss_Bz / len(train_loader)
    avg_physics_loss = epoch_physics_loss / len(train_loader)

    train_losses_Bx.append(avg_loss_Bx)
    train_losses_By.append(avg_loss_By)
    train_losses_Bz.append(avg_loss_Bz)
    physics_losses.append(avg_physics_loss)

    model.eval()
    epoch_r2_Bx = []
    epoch_r2_By = []
    epoch_r2_Bz = []
    test_loader_tqdm = tqdm(test_loader, total=len(test_loader), desc=f'Test Epoch {epoch+1}/{num_epochs}')
    with torch.no_grad():
        for sample in test_loader_tqdm:
            x = sample['input'].to(device0)
            lengths = sample['lengths'].to(device0)  
            target = sample['output'].cpu().numpy()
            output = model(x, lengths).cpu().numpy()

            for true_sample, pred_sample in zip(target, output):
                r2_Bx = r2_score(true_sample[:, 0].flatten(), pred_sample[:, 0].flatten())
                r2_By = r2_score(true_sample[:, 1].flatten(), pred_sample[:, 1].flatten())
                r2_Bz = r2_score(true_sample[:, 2].flatten(), pred_sample[:, 2].flatten())

                epoch_r2_Bx.append(r2_Bx)
                epoch_r2_By.append(r2_By)
                epoch_r2_Bz.append(r2_Bz)

    avg_r2_Bx = np.mean(epoch_r2_Bx)
    avg_r2_By = np.mean(epoch_r2_By)
    avg_r2_Bz = np.mean(epoch_r2_Bz)

    test_losses_Bx.append(avg_r2_Bx)
    test_losses_By.append(avg_r2_By)
    test_losses_Bz.append(avg_r2_Bz)

    end_time = time.time()
    epoch_duration = end_time - start_time
    epoch_durations.append(epoch_duration)

    avg_train_loss_combined = (avg_loss_Bx + avg_loss_By + avg_loss_Bz + avg_physics_loss) / 3
    scheduler.step(avg_train_loss_combined)

    avg_epoch_duration = np.mean(epoch_durations)
    remaining_epochs = num_epochs - (epoch + 1)
    remaining_time = remaining_epochs * avg_epoch_duration
    hours, rem = divmod(remaining_time, 3600)
    minutes, seconds = divmod(rem, 60)

    print(f'Epoch: {epoch+1}/{num_epochs}, Duration: {epoch_duration:.2f}s, '
          f'Train Loss - Bx: {avg_loss_Bx:.4f}, By: {avg_loss_By:.4f}, Bz: {avg_loss_Bz:.4f}, Physics Loss: {avg_physics_loss:.4f}, '
          f'Test R2 - Bx: {avg_r2_Bx:.4f}, By: {avg_r2_By:.4f}, Bz: {avg_r2_Bz:.4f}')

    print(f'Estimated Remaining Time: {int(hours)}h {int(minutes)}m {int(seconds)}s')

    if (epoch + 1) >= save_checkpoint_start_epoch and (epoch + 1) % save_checkpoint_interval == 0:
        checkpoint_path = os.path.join(checkpoint_dir, f"checkpoint_epoch_{epoch+1}.pt")
        torch.save(model.state_dict(), checkpoint_path)
        print(f"Checkpoint saved at epoch {epoch+1}: {checkpoint_path}")

        min_val = float('inf')
        max_val = float('-inf')
        for sample in test_loader:
            y_true_batch = sample['output'].numpy()
            batch_min = y_true_batch.min()
            batch_max = y_true_batch.max()
            min_val = min(min_val, batch_min)
            max_val = max(max_val, batch_max)
        data_range = max_val - min_val
        print(f"[INFO] data_range = {data_range} (max_val={max_val}, min_val={min_val})")

        type_performance = {}

        sample_r2_scores = {'bx': [], 'by': [], 'bz': []}
        sample_relative_errors = {'bx': [], 'by': [], 'bz': []}
        sample_mses = {'bx': [], 'by': [], 'bz': []}
        sample_maes = {'bx': [], 'by': [], 'bz': []}
        sample_psnr_values = {'bx': [], 'by': [], 'bz': []}
        sample_ssim_values = {'bx': [], 'by': [], 'bz': []}

        model.eval()
        with torch.no_grad():
            for sample in test_loader:
                x = sample['input'].to(device0)
                lengths = sample['lengths'].to(device0)  
                y_true_batch = sample['output'].cpu().numpy()
                y_pred_batch = model(x, lengths).cpu().numpy()

                for true_sample, pred_sample in zip(y_true_batch, y_pred_batch):
                    for i, comp in enumerate(['bx', 'by', 'bz']):
                        true_sample_component = true_sample[:, i]
                        pred_sample_component = pred_sample[:, i]

                        true_sample_flat = true_sample_component.flatten()
                        pred_sample_flat = pred_sample_component.flatten()

                        r2 = r2_score(true_sample_flat, pred_sample_flat)
                        sample_r2_scores[comp].append(r2)

                        absolute_error = np.linalg.norm(true_sample_flat - pred_sample_flat, 1)
                        relative_error = absolute_error / np.linalg.norm(true_sample_flat, 1)
                        sample_relative_errors[comp].append(relative_error)

                        mse = mean_squared_error(true_sample_flat, pred_sample_flat)
                        sample_mses[comp].append(mse)

                        mae = mean_absolute_error(true_sample_flat, pred_sample_flat)
                        sample_maes[comp].append(mae)

                        psnr_value = psnr(true_sample_component, pred_sample_component, data_range=data_range)
                        sample_psnr_values[comp].append(psnr_value)

                        ssim_value = ssim(true_sample_component, pred_sample_component, data_range=data_range)
                        sample_ssim_values[comp].append(ssim_value)
                        
        with open(results_file, mode='a', newline='') as file:
            writer = csv.writer(file)
            for comp in ['bx', 'by', 'bz']:
                writer.writerow([epoch+1, comp,
                                 np.mean(sample_r2_scores[comp]),
                                 np.mean(sample_relative_errors[comp]),
                                 np.mean(sample_mses[comp]),
                                 np.mean(sample_maes[comp]),
                                 np.mean(sample_psnr_values[comp]),
                                 np.mean(sample_ssim_values[comp])])

        model.eval()
        with torch.no_grad():
            for sample in test_loader:
                x = sample['input'].to(device0)
                lengths = sample['lengths'].to(device0)  
                y_true_batch = sample['output'].cpu().numpy()
                y_pred_batch = model(x, lengths).cpu().numpy()

                type_labels = sample['type_label_str']  

                for true_sample, pred_sample, t_str in zip(y_true_batch, y_pred_batch, type_labels):
                    if t_str not in type_performance:
                        type_performance[t_str] = {
                            'bx': {'r2': [], 'mse': [], 'mae': [], 'psnr': [], 'ssim': [], 'rel_err': []},
                            'by': {'r2': [], 'mse': [], 'mae': [], 'psnr': [], 'ssim': [], 'rel_err': []},
                            'bz': {'r2': [], 'mse': [], 'mae': [], 'psnr': [], 'ssim': [], 'rel_err': []},
                        }

                    for i, comp in enumerate(['bx', 'by', 'bz']):
                        true_comp = true_sample[:, i].flatten()
                        pred_comp = pred_sample[:, i].flatten()
                        r2_val = r2_score(true_comp, pred_comp)

                        abs_err = np.linalg.norm(true_comp - pred_comp, 1)
                        rel_err = abs_err / np.linalg.norm(true_comp, 1)

                        mse_val = mean_squared_error(true_comp, pred_comp)

                        mae_val = mean_absolute_error(true_comp, pred_comp)

                        psnr_val = psnr(true_comp, pred_comp, data_range=data_range)
                        ssim_val = ssim(
                            true_comp,
                            pred_comp,
                            data_range=data_range
                        )

                        metrics = type_performance[t_str][comp]
                        metrics['r2'].append(r2_val)
                        metrics['rel_err'].append(rel_err)
                        metrics['mse'].append(mse_val)
                        metrics['mae'].append(mae_val)
                        metrics['psnr'].append(psnr_val)
                        metrics['ssim'].append(ssim_val)
                        
        avg_type_performance = {}
        for t_str, comp_dict in type_performance.items():
            avg_type_performance[t_str] = {}
            for comp in ['bx', 'by', 'bz']:
                avg_type_performance[t_str][comp] = {
                    'r2':    np.mean(comp_dict[comp]['r2']),
                    'mse':   np.mean(comp_dict[comp]['mse']),
                    'mae':   np.mean(comp_dict[comp]['mae']),
                    'psnr':  np.mean(comp_dict[comp]['psnr']),
                    'ssim':  np.mean(comp_dict[comp]['ssim']),
                    'rel_err': np.mean(comp_dict[comp]['rel_err']),
                }

        all_types = sorted(avg_type_performance.keys())
        comps = ['bx', 'by', 'bz']
        
        scores_by_comp = []
        for comp in comps:
            comp_scores = [avg_type_performance[t][comp]['r2'] for t in all_types]
            scores_by_comp.append(comp_scores)
        scores_by_comp = np.array(scores_by_comp)  # shape: (3, n_types)

        plt.figure(figsize=(10,6))
        bar_width = 0.25
        x = np.arange(len(all_types))

        for i, comp in enumerate(comps):
            plt.bar(
                x + i*bar_width, 
                scores_by_comp[i], 
                width=bar_width, 
                label=comp.upper()
            )

        plt.xticks(x + bar_width, all_types, rotation=45)
        plt.ylabel('R2 Score')
        

        ax = plt.gca()

        ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.4f'))

        plt.title(f'Comparison of R2 by Type (Epoch {epoch+1})')
        plt.legend()
        plt.tight_layout()
        
        plot_path = os.path.join(checkpoint_dir, f"R2_bar_chart_epoch_{epoch+1}.png")
        plt.savefig(plot_path)
        plt.close()
        print(f"Grouped Bar Chart saved: {plot_path}")

np.save(os.path.join(checkpoint_dir, 'train_losses_Bx.npy'), np.array(train_losses_Bx))
np.save(os.path.join(checkpoint_dir, 'train_losses_By.npy'), np.array(train_losses_By))
np.save(os.path.join(checkpoint_dir, 'train_losses_Bz.npy'), np.array(train_losses_Bz))
np.save(os.path.join(checkpoint_dir, 'physics_losses.npy'), np.array(physics_losses))
np.save(os.path.join(checkpoint_dir, 'test_losses_Bx.npy'), np.array(test_losses_Bx))
np.save(os.path.join(checkpoint_dir, 'test_losses_By.npy'), np.array(test_losses_By))
np.save(os.path.join(checkpoint_dir, 'test_losses_Bz.npy'), np.array(test_losses_Bz))

plt.figure(figsize=(10, 5))
plt.plot(train_losses_Bx, label='Training Loss Bx', color='red')
plt.plot(train_losses_By, label='Training Loss By', color='green')
plt.plot(train_losses_Bz, label='Training Loss Bz', color='blue')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Losses for Bx, By, Bz')
plt.legend()
plt.show()

plt.figure(figsize=(10, 5))
plt.plot(test_losses_Bx, label='Testing R2 Bx', color='red')
plt.plot(test_losses_By, label='Testing R2 By', color='green')
plt.plot(test_losses_Bz, label='Testing R2 Bz', color='blue')
plt.xlabel('Epoch')
plt.ylabel('R2 Score')
plt.title('Testing R2 Scores for Bx, By, Bz')
plt.legend()
plt.show()


In [ ]:
model.load_state_dict(torch.load("result/NLFFF_Cube2Step/step1/checkpoint_epoch_150.pt"))
model.eval() 

In [ ]:
import torch


all_output_Bx = []
all_output_By = []
all_output_Bz = []
all_target_Bx = []
all_target_By = []
all_target_Bz = []

for sample in train_loader:
    input_data = sample['input'].to(device0)  
    lengths = sample['lengths'].to(device0)  

    with torch.no_grad():
        output = model(input_data, lengths)

    output_Bx = output[:, :, 0, :, :].cpu().numpy() 
    output_By = output[:, :, 1, :, :].cpu().numpy()
    output_Bz = output[:, :, 2, :, :].cpu().numpy()  

    target_Bx = sample['output'][:, :, 0, :, :].cpu().numpy()  
    target_By = sample['output'][:, :, 1, :, :].cpu().numpy()  
    target_Bz = sample['output'][:, :, 2, :, :].cpu().numpy() 

    all_output_Bx.append(output_Bx)
    all_output_By.append(output_By)
    all_output_Bz.append(output_Bz)

    all_target_Bx.append(target_Bx)
    all_target_By.append(target_By)
    all_target_Bz.append(target_Bz)

all_output_Bx = np.concatenate(all_output_Bx, axis=0)
all_output_By = np.concatenate(all_output_By, axis=0)
all_output_Bz = np.concatenate(all_output_Bz, axis=0)
all_target_Bx = np.concatenate(all_target_Bx, axis=0)
all_target_By = np.concatenate(all_target_By, axis=0)
all_target_Bz = np.concatenate(all_target_Bz, axis=0)

print(f'All Bx shape: {all_output_Bx.shape}')
print(f'All By shape: {all_output_By.shape}')
print(f'All Bz shape: {all_output_Bz.shape}')
print(f'All target By shape: {all_target_By.shape}')

all_test_output_Bx = []
all_test_output_By = []
all_test_output_Bz = []
all_test_target_Bx = []
all_test_target_By = []
all_test_target_Bz = []


for sample in test_loader:

    input_data = sample['input'].to(device0) 
    lengths = sample['lengths'].to(device0)

    with torch.no_grad():
        output = model(input_data, lengths)

    output_Bx = output[:, :, 0, :, :].cpu().numpy()
    output_By = output[:, :, 1, :, :].cpu().numpy()
    output_Bz = output[:, :, 2, :, :].cpu().numpy()  

    target_Bx = sample['output'][:, :, 0, :, :].cpu().numpy()  
    target_By = sample['output'][:, :, 1, :, :].cpu().numpy()  
    target_Bz = sample['output'][:, :, 2, :, :].cpu().numpy()  

    all_test_output_Bx.append(output_Bx)
    all_test_output_By.append(output_By)
    all_test_output_Bz.append(output_Bz)
    all_test_target_Bx.append(target_Bx)
    all_test_target_By.append(target_By)
    all_test_target_Bz.append(target_Bz)

all_test_output_Bx = np.concatenate(all_test_output_Bx, axis=0)
all_test_output_By = np.concatenate(all_test_output_By, axis=0)
all_test_output_Bz = np.concatenate(all_test_output_Bz, axis=0)
all_test_target_Bx = np.concatenate(all_test_target_Bx, axis=0)
all_test_target_By = np.concatenate(all_test_target_By, axis=0)
all_test_target_Bz = np.concatenate(all_test_target_Bz, axis=0)

combined_output_Bx = np.concatenate((all_output_Bx, all_test_output_Bx), axis=0)
combined_output_By = np.concatenate((all_output_By, all_test_output_By), axis=0)
combined_output_Bz = np.concatenate((all_output_Bz, all_test_output_Bz), axis=0)

combined_target_Bx = np.concatenate((all_target_Bx, all_test_target_Bx), axis=0)
combined_target_By = np.concatenate((all_target_By, all_test_target_By), axis=0)
combined_target_Bz = np.concatenate((all_target_Bz, all_test_target_Bz), axis=0)

print(f'All Test Bx shape: {combined_output_Bx.shape}')
print(f'All Test By shape: {combined_output_By.shape}')
print(f'All Test Bz shape: {combined_output_Bz.shape}')

print(f'All Test target By shape: {combined_target_By.shape}')

In [ ]:
import torch
from torch.utils.data import Dataset
import numpy as np

class NewCustomDataset(Dataset):
    
    def __init__(
        self, 
        Bx_data, By_data, Bz_data, 
        Bx_targets, By_targets, Bz_targets, 
        x_lengths, y_lengths, z_lengths,
        type_labels=None
    ):
        
        self.Bx_data = Bx_data
        self.By_data = By_data
        self.Bz_data = Bz_data

        self.Bx_targets = Bx_targets
        self.By_targets = By_targets
        self.Bz_targets = Bz_targets
        
        self.x_lengths = x_lengths
        self.y_lengths = y_lengths
        self.z_lengths = z_lengths

        self.x_mean = np.mean(self.x_lengths)
        self.x_std = np.std(self.x_lengths)

        self.y_mean = np.mean(self.y_lengths)
        self.y_std = np.std(self.y_lengths)

        self.z_mean = np.mean(self.z_lengths)
        self.z_std = np.std(self.z_lengths)

        self.type_labels = type_labels  

    def __len__(self):
        return self.Bx_data.shape[0]

    def __getitem__(self, idx):

        input_Bx = self.Bx_data[idx]  
        input_By = self.By_data[idx]
        input_Bz = self.Bz_data[idx]  

        input_data = torch.stack((input_Bx, input_By, input_Bz), dim=1)  

        target_Bx = self.Bx_targets[idx]
        target_By = self.By_targets[idx]
        target_Bz = self.Bz_targets[idx]

        target_data = torch.stack((target_Bx, target_By, target_Bz), dim=1)

        x_length = self.x_lengths[idx]
        y_length = self.y_lengths[idx]
        z_length = self.z_lengths[idx]

        x_length_normalized = (x_length - self.x_mean) / self.x_std
        y_length_normalized = (y_length - self.y_mean) / self.y_std
        z_length_normalized = (z_length - self.z_mean) / self.z_std

        length_tensor = torch.tensor(
            [x_length_normalized, y_length_normalized, z_length_normalized],
            dtype=torch.float32
        ).unsqueeze(0)

        type_label = None
        if self.type_labels is not None:
            type_label = self.type_labels[idx]
        
        sample = {
            'input': input_data,           
            'lengths': length_tensor,
            'target': target_data
        }

        if type_label is not None:
            sample['type_label'] = type_label  

        return sample

x_lengths = np.load("NLFFF_data/x_height.npy")
y_lengths = np.load("NLFFF_data/y_height.npy")
z_lengths = np.load("NLFFF_data/z_height.npy")

combined_output_Bx = torch.tensor(combined_output_Bx).float()
combined_output_By = torch.tensor(combined_output_By).float()
combined_output_Bz = torch.tensor(combined_output_Bz).float()

combined_target_Bx = torch.tensor(combined_target_Bx).float()
combined_target_By = torch.tensor(combined_target_By).float()
combined_target_Bz = torch.tensor(combined_target_Bz).float()

df = pd.read_excel("NLFFF_data/NLFFF_datatype.xlsx")
type_labels_from_excel = df["type"].values

dataset = NewCustomDataset(combined_output_Bx, combined_output_By, combined_output_Bz, combined_target_Bx, combined_target_By, combined_target_Bz, x_lengths, y_lengths, z_lengths, type_labels_from_excel)

train_size = 143
test_size = 27

train_indices = list(range(train_size))
test_indices = list(range(train_size, train_size + test_size))

train_dataset = torch.utils.data.Subset(dataset, train_indices)
test_dataset = torch.utils.data.Subset(dataset, test_indices)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=10, shuffle=True, num_workers=8, persistent_workers=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=10, shuffle=False, num_workers=8, persistent_workers=False)

for batch in train_loader:
    print(batch['input'].shape)  
    print(batch['lengths'].shape)
    print(batch['target'].shape)
    break


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class PhysicsInformedLoss(nn.Module):
    def __init__(self, 
                 weight=1.0, 
                 beta=1.0,    
                 alpha=1.0,   
                 gamma=1.0,   
                 delta=1.0,   
                 ):
        
        super(PhysicsInformedLoss, self).__init__()
        self.weight = weight
        self.beta = beta       
        self.gamma = gamma    
        self.delta = delta     
        self.alpha = alpha     

    def forward(self, output, target):

        divergence_loss = self.compute_divergence_loss(output)
        force_free_loss = self.compute_force_free_loss(output)
        grad_diff_loss = self.compute_gradient_difference(output, target)
        free_energy_loss = self.compute_free_energy_loss(output, target)

        total_loss = self.weight * (
            self.beta * divergence_loss +
            self.gamma * force_free_loss +
            self.alpha * free_energy_loss
        )
        return total_loss

    def compute_divergence_loss(self, output):
    
        batch_size, depth, components, height, width = output.shape
        B = output.permute(0, 2, 1, 3, 4)  # [batch_size, 3, depth, height, width]
        channels = components  

        device = output.device
        kernel_dx = torch.tensor([-0.5, 0, 0.5], device=device).reshape(1, 1, 1, 1, 3)
        kernel_dy = torch.tensor([-0.5, 0, 0.5], device=device).reshape(1, 1, 1, 3, 1)
        kernel_dz = torch.tensor([-0.5, 0, 0.5], device=device).reshape(1, 1, 3, 1, 1)

        kernel_dx = kernel_dx.repeat(channels, 1, 1, 1, 1)
        kernel_dy = kernel_dy.repeat(channels, 1, 1, 1, 1)
        kernel_dz = kernel_dz.repeat(channels, 1, 1, 1, 1)

        dB_dx = F.conv3d(B, kernel_dx, padding=(0, 0, 1), groups=channels)
        dB_dy = F.conv3d(B, kernel_dy, padding=(0, 1, 0), groups=channels)
        dB_dz = F.conv3d(B, kernel_dz, padding=(1, 0, 0), groups=channels)

        dB_x_dx = dB_dx[:, 0, :, :, :]
        dB_y_dy = dB_dy[:, 1, :, :, :]
        dB_z_dz = dB_dz[:, 2, :, :, :]

        divergence = dB_x_dx + dB_y_dy + dB_z_dz
        divergence_loss = torch.mean(divergence ** 2)

        return divergence_loss

    def compute_force_free_loss(self, output):
        # output: [batch_size, z, 3, y, x]
        batch_size, depth, components, height, width = output.shape
        B = output.permute(0, 2, 1, 3, 4)  # [batch_size, 3, depth, height, width]
        channels = components

        device = output.device
        kernel_dx = torch.tensor([-0.5, 0, 0.5], device=device).reshape(1, 1, 1, 1, 3)
        kernel_dy = torch.tensor([-0.5, 0, 0.5], device=device).reshape(1, 1, 1, 3, 1)
        kernel_dz = torch.tensor([-0.5, 0, 0.5], device=device).reshape(1, 1, 3, 1, 1)

        kernel_dx = kernel_dx.repeat(channels, 1, 1, 1, 1)
        kernel_dy = kernel_dy.repeat(channels, 1, 1, 1, 1)
        kernel_dz = kernel_dz.repeat(channels, 1, 1, 1, 1)

        dB_dx = F.conv3d(B, kernel_dx, padding=(0, 0, 1), groups=channels)
        dB_dy = F.conv3d(B, kernel_dy, padding=(0, 1, 0), groups=channels)
        dB_dz = F.conv3d(B, kernel_dz, padding=(1, 0, 0), groups=channels)

        dB_x_dy = dB_dy[:, 0, :, :, :]
        dB_x_dz = dB_dz[:, 0, :, :, :]
        dB_y_dx = dB_dx[:, 1, :, :, :]
        dB_y_dz = dB_dz[:, 1, :, :, :]
        dB_z_dx = dB_dx[:, 2, :, :, :]
        dB_z_dy = dB_dy[:, 2, :, :, :]

        rot_x = dB_z_dy - dB_y_dz
        rot_y = dB_x_dz - dB_z_dx
        rot_z = dB_y_dx - dB_x_dy

        j = torch.stack([rot_x, rot_y, rot_z], dim=1)

        jxb = torch.cross(j, B, dim=1)

        B_magnitude_squared = torch.sum(B ** 2, dim=1, keepdim=True) + 1e-7

        force_free_loss = torch.mean(torch.sum(jxb ** 2, dim=1) / B_magnitude_squared.squeeze(1))
        return force_free_loss

    def compute_gradient_difference(self, output, target):
        if target.shape != output.shape:
            return torch.tensor(0.0, device=output.device)

        batch_size, depth, components, height, width = output.shape
        B_pred = output.permute(0, 2, 1, 3, 4)   # [batch_size, 3, depth, height, width]
        B_true = target.permute(0, 2, 1, 3, 4)
        channels = components

        device = output.device
        kernel_dx = torch.tensor([-0.5, 0, 0.5], device=device).reshape(1, 1, 1, 1, 3)
        kernel_dy = torch.tensor([-0.5, 0, 0.5], device=device).reshape(1, 1, 1, 3, 1)
        kernel_dz = torch.tensor([-0.5, 0, 0.5], device=device).reshape(1, 1, 3, 1, 1)

        kernel_dx = kernel_dx.repeat(channels, 1, 1, 1, 1)
        kernel_dy = kernel_dy.repeat(channels, 1, 1, 1, 1)
        kernel_dz = kernel_dz.repeat(channels, 1, 1, 1, 1)

        dBp_dx = F.conv3d(B_pred, kernel_dx, padding=(0, 0, 1), groups=channels)
        dBp_dy = F.conv3d(B_pred, kernel_dy, padding=(0, 1, 0), groups=channels)
        dBp_dz = F.conv3d(B_pred, kernel_dz, padding=(1, 0, 0), groups=channels)

        dBt_dx = F.conv3d(B_true, kernel_dx, padding=(0, 0, 1), groups=channels)
        dBt_dy = F.conv3d(B_true, kernel_dy, padding=(0, 1, 0), groups=channels)
        dBt_dz = F.conv3d(B_true, kernel_dz, padding=(1, 0, 0), groups=channels)

        grad_diff = (dBp_dx - dBt_dx)**2 + \
                    (dBp_dy - dBt_dy)**2 + \
                    (dBp_dz - dBt_dz)**2

        loss_val = torch.mean(grad_diff)
        return loss_val

    def compute_free_energy_loss(self, output, target):
    
        if output.shape != target.shape:
            return torch.tensor(0.0, device=output.device)

        B_pred = output.permute(0, 2, 1, 3, 4)
        B_true = target.permute(0, 2, 1, 3, 4)

        B_diff = B_pred - B_true 

        B_diff_sq = (B_diff ** 2).sum(dim=(1,2,3,4))     
        E_diff = B_diff_sq / (8.0 * math.pi)          

        free_energy_loss = torch.mean(E_diff)/1e5

        return free_energy_loss

class singlePhysicsInformedDivergence_Loss(nn.Module):
    def __init__(self, weight=1.0, alpha=1.0, beta=1.0, gamma=1.0):
        super(singlePhysicsInformedDivergence_Loss, self).__init__()
        self.weight = weight
        self.alpha = alpha  
        self.beta = beta    
        self.gamma = gamma  

    def forward(self, output, target): 
        loss = self.compute_single_slice_physics_loss(output)
        
        return loss

    def compute_single_slice_physics_loss(self, output_slice):

        batch_size, components, height, width = output_slice.shape
        #print(output_slice.shape)
        B = output_slice.unsqueeze(2)
        device = output_slice.device

        kernel_dx = torch.tensor([-0.5, 0, 0.5], device=device).reshape(1, 1, 1, 1, 3)
        kernel_dy = torch.tensor([-0.5, 0, 0.5], device=device).reshape(1, 1, 1, 3, 1)
        kernel_dz = torch.tensor([-0.5, 0, 0.5], device=device).reshape(1, 1, 3, 1, 1)

        kernel_dx = kernel_dx.repeat(components, 1, 1, 1, 1)
        kernel_dy = kernel_dy.repeat(components, 1, 1, 1, 1)
        kernel_dz = kernel_dz.repeat(components, 1, 1, 1, 1)

        dB_dx = F.conv3d(B, kernel_dx, padding=(0, 0, 1), groups=components)
        dB_dy = F.conv3d(B, kernel_dy, padding=(0, 1, 0), groups=components)
        dB_dz = F.conv3d(B, kernel_dz, padding=(1, 0, 0), groups=components)

        dB_x_dx = dB_dx[:, 0, :, :]
        dB_y_dy = dB_dy[:, 1, :, :]
        dB_z_dz = dB_dz[:, 2, :, :]

        divergence = dB_x_dx + dB_y_dy + dB_z_dz
        divergence_loss = torch.mean(divergence ** 2, dim=(1, 2, 3))

        dB_x_dy = dB_dy[:, 0, :, :]
        dB_x_dz = dB_dz[:, 0, :, :]

        dB_y_dx = dB_dx[:, 1, :, :]
        dB_y_dz = dB_dz[:, 1, :, :]

        dB_z_dx = dB_dx[:, 2, :, :]
        dB_z_dy = dB_dy[:, 2, :, :]

        rot_x = dB_z_dy - dB_y_dz
        rot_y = dB_x_dz - dB_z_dx
        rot_z = dB_y_dx - dB_x_dy

        j = torch.stack([rot_x, rot_y, rot_z], dim=1)  # [batch_size, 3, height, width]

        jxb = torch.cross(j, B, dim=1)  # [batch_size, 3, height, width]
        B_magnitude_squared = torch.sum(B ** 2, dim=1, keepdim=True) + 1e-7
        force_free_loss = torch.mean(torch.sum(jxb ** 2, dim=1) / B_magnitude_squared.squeeze(1), dim=(1, 2, 3))

        physics_loss = divergence_loss

        return physics_loss

class singlePhysicsInformedForce_Free_Loss(nn.Module):
    def __init__(self, weight=1.0, alpha=1.0, beta=1.0, gamma=1.0):
        super(singlePhysicsInformedForce_Free_Loss, self).__init__()
        self.weight = weight
        self.alpha = alpha  
        self.beta = beta    
        self.gamma = gamma  

    def forward(self, output, target): 
        loss = self.compute_single_slice_physics_loss(output)
        
        return loss

    def compute_single_slice_physics_loss(self, output_slice):

        batch_size, components, height, width = output_slice.shape
        #print(output_slice.shape)
        B = output_slice.unsqueeze(2)
        device = output_slice.device

        kernel_dx = torch.tensor([-0.5, 0, 0.5], device=device).reshape(1, 1, 1, 1, 3)
        kernel_dy = torch.tensor([-0.5, 0, 0.5], device=device).reshape(1, 1, 1, 3, 1)
        kernel_dz = torch.tensor([-0.5, 0, 0.5], device=device).reshape(1, 1, 3, 1, 1)

        kernel_dx = kernel_dx.repeat(components, 1, 1, 1, 1)
        kernel_dy = kernel_dy.repeat(components, 1, 1, 1, 1)
        kernel_dz = kernel_dz.repeat(components, 1, 1, 1, 1)

        dB_dx = F.conv3d(B, kernel_dx, padding=(0, 0, 1), groups=components)
        dB_dy = F.conv3d(B, kernel_dy, padding=(0, 1, 0), groups=components)
        dB_dz = F.conv3d(B, kernel_dz, padding=(1, 0, 0), groups=components)

        dB_x_dx = dB_dx[:, 0, :, :]

        dB_y_dy = dB_dy[:, 1, :, :]

        dB_z_dz = dB_dz[:, 2, :, :]

        divergence = dB_x_dx + dB_y_dy + dB_z_dz
        divergence_loss = torch.mean(divergence ** 2, dim=(1, 2, 3))

        dB_x_dy = dB_dy[:, 0, :, :]
        dB_x_dz = dB_dz[:, 0, :, :]

        dB_y_dx = dB_dx[:, 1, :, :]
        dB_y_dz = dB_dz[:, 1, :, :]

        dB_z_dx = dB_dx[:, 2, :, :]
        dB_z_dy = dB_dy[:, 2, :, :]

        rot_x = dB_z_dy - dB_y_dz
        rot_y = dB_x_dz - dB_z_dx
        rot_z = dB_y_dx - dB_x_dy

        j = torch.stack([rot_x, rot_y, rot_z], dim=1)  # [batch_size, 3, height, width]

        jxb = torch.cross(j, B, dim=1)  # [batch_size, 3, height, width]
        B_magnitude_squared = torch.sum(B ** 2, dim=1, keepdim=True) + 1e-7
        force_free_loss = torch.mean(torch.sum(jxb ** 2, dim=1) / B_magnitude_squared.squeeze(1), dim=(1, 2, 3))

        physics_loss = force_free_loss

        return physics_loss


class ECA3DLayer(nn.Module):
    def __init__(self, channel, k_size=3):
        super(ECA3DLayer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool3d(1)
        self.conv = nn.Conv1d(1, 1, kernel_size=k_size, padding=(k_size - 1) // 2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # x: [batch_size, channels, seq_len, height, width]
        
        # Global average pooling across seq_len, height, width
        y = self.avg_pool(x)  # Shape: [batch_size, channels, 1, 1, 1]

        # Apply 1D convolution across the channel dimension
        y = self.conv(y.squeeze(-1).squeeze(-1).transpose(-1, -2))  # Shape: [batch_size, 1, channels]
        y = y.transpose(-1, -2).unsqueeze(-1).unsqueeze(-1)  # Shape: [batch_size, channels, 1, 1, 1]

        # Apply attention to input
        return x * self.sigmoid(y).expand_as(x)


class DilatedConvYWithECA(nn.Module):
    def __init__(self, channels, dilation_y=2, k_size=3):
        super(DilatedConvYWithECA, self).__init__()
        # Dilated convolution in y dimension
        self.conv = nn.Conv3d(
            in_channels=channels,
            out_channels=channels,
            kernel_size=(1, 3, 1),  
            padding=(0, dilation_y, 0),  
            dilation=(1, dilation_y, 1),
            bias=False
        )
        self.activation = nn.ReLU()

        # ECA Attention layer
        self.eca = ECA3DLayer(channel=channels, k_size=k_size)

    def forward(self, x):
        # Apply dilated convolution
        x = self.conv(x)
        x = self.activation(x)

        # Apply ECA attention
        x = self.eca(x)
        
        return x


class CustomFNOnew(nn.Module):
    def __init__(self, n_modes, hidden_channels, in_channels=3, out_channels=1,
                 lifting_channels=256, projection_channels=256, n_layers=4,
                 output_scaling_factor=None, max_n_modes=None,
                 fno_block_precision="full", use_mlp=False, mlp_dropout=0,
                 mlp_expansion=0.5, non_linearity=F.gelu, stabilizer=None,
                 norm=None, preactivation=False, fno_skip="linear",
                 mlp_skip="soft-gating", separable=False, factorization=None,
                 rank=1.0, joint_factorization=False, fixed_rank_modes=False,
                 implementation="factorized", decomposition_kwargs=dict(),
                 domain_padding=None, domain_padding_mode="one-sided",
                 fft_norm="forward", SpectralConv=SpectralConv,
                 **kwargs):
        
        super().__init__()
        self.n_dim = len(n_modes)
        self.n_layers = n_layers

        # Define the ECA-Net block for lengths_lifted
        self.DilatedConvYWithECA = DilatedConvYWithECA(hidden_channels, dilation_y=2).to(device0)

        # Define the lifting layers for two inputs
        self.lifting1 = MLP(
            in_channels=3,
            out_channels=hidden_channels,
            hidden_channels=lifting_channels,
            n_layers=1,
            n_dim=self.n_dim,
        ).to(device0)
        
        self.lifting_y = MLP(
            in_channels=3,  
            out_channels=hidden_channels,
            hidden_channels=lifting_channels,
            n_layers=2,
            n_dim=1,
        ).to(device0)
        
    
        # Define the FNO blocks
        self.fno_blocks = FNOBlocks(
            in_channels=hidden_channels,
            out_channels=hidden_channels,
            n_modes=n_modes,
            output_scaling_factor=output_scaling_factor,
            use_mlp=use_mlp,
            mlp_dropout=mlp_dropout,
            mlp_expansion=mlp_expansion,
            non_linearity=non_linearity,
            stabilizer=stabilizer,
            norm=norm,
            preactivation=preactivation,
            fno_skip=fno_skip,
            mlp_skip=mlp_skip,
            max_n_modes=max_n_modes,
            fno_block_precision=fno_block_precision,
            rank=rank,
            fft_norm=fft_norm,
            fixed_rank_modes=fixed_rank_modes,
            implementation=implementation,
            separable=separable,
            factorization=factorization,
            decomposition_kwargs=decomposition_kwargs,
            joint_factorization=joint_factorization,
            SpectralConv=SpectralConv,
            n_layers=n_layers,
            **kwargs
        ).to(device0)

        self.projection = MLP(
            in_channels=hidden_channels,
            out_channels=out_channels,
            hidden_channels=projection_channels,
            n_layers=2,
            n_dim=self.n_dim,
            non_linearity=non_linearity,
        ).to(device0)

    
    def forward(self, x, lengths):

        x_orig = x.clone()
        
        lengths = lengths.view(-1, 3, 1)  

        lengths_lifted = self.lifting_y(lengths)  

        lengths_lifted = lengths_lifted.unsqueeze(-1).unsqueeze(-1)

        lengths_lifted = self.DilatedConvYWithECA(lengths_lifted)
        
        x = x + lengths_lifted

        for layer_idx in range(self.n_layers):
            x = self.fno_blocks(x, layer_idx)

        x = self.projection(x)

        x[:, 0, :, :, :] = x_orig[:, 0, :, :, :]

        return x


In [ ]:
import argparse
import torch
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP

newmodel = CustomFNOnew(n_modes = (32, 32, 32), in_channels=1, out_channels=25, n_layers = 6,
               hidden_channels=25, use_mlp=True, factorization='tucker', rank=0.2)

newmodel = newmodel.to(device0)

n_params = count_model_params(newmodel)
print(f'\nYour model has {n_params} parameters.')

optimizer = torch.optim.Adam(newmodel.parameters(), lr=1e-3, weight_decay=1e-4) 

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.8, patience=1, verbose=True)

h1loss = H1Loss(d=2)

In [ ]:
import os
import time
import csv
import torch.nn.functional as F
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from skimage.metrics import peak_signal_noise_ratio as psnr, structural_similarity as ssim
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

num_epochs = 150
print_frequency = 40
save_checkpoint_start_epoch = 20
save_checkpoint_interval = 10
checkpoint_dir = "result/NLFFF_Cube2Step/step2"
results_file = os.path.join(checkpoint_dir, "result.csv")

physics_loss_fn = PhysicsInformedLoss(weight=0.35, alpha=1, beta=1, gamma=1, delta = 0)
single_physics_loss_DIV_fn = singlePhysicsInformedDivergence_Loss(weight=1, alpha=1.0, beta=1, gamma=1)
single_physics_loss_FF_fn = singlePhysicsInformedForce_Free_Loss(weight=1, alpha=1.0, beta=1, gamma=1)

with open(results_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['epoch', 'comp', 'r2', 'relative_error', 'mse', 'mae', 'psnr', 'ssim'])

train_losses_Bx = []
train_losses_By = []
train_losses_Bz = []
test_losses_Bx = []
test_losses_By = []
test_losses_Bz = []
physics_losses_epoch = []
epoch_durations = []

#z_df = pd.read_csv(z_csv_path)
#z_values = z_df['z'].values
#assert len(z_values) == 25

for epoch in range(num_epochs):
    start_time = time.time()

    model.train()
    epoch_loss_Bx = 0.0
    epoch_loss_By = 0.0
    epoch_loss_Bz = 0.0
    epoch_physics_loss = 0.0
    
    
    time_step_DIV_losses_accum = None
    time_step_FF_losses_accum = None

    train_loader_tqdm = tqdm(enumerate(train_loader), total=len(train_loader), desc=f'Train Epoch {epoch+1}/{num_epochs}')
    for batch_idx, (batch_i, sample) in enumerate(train_loader_tqdm):
        optimizer.zero_grad()
        data, target = sample['input'].to(device0), sample['target'].to(device0)
        lengths = sample['lengths'].to(device0)  

        output = newmodel(data, lengths)

        #print(output.shape)

        loss_Bx = h1loss(output[:, :, 0, :, :], target[:, :, 0, :, :]).mean()
        loss_By = h1loss(output[:, :, 1, :, :], target[:, :, 1, :, :]).mean()
        loss_Bz = h1loss(output[:, :, 2, :, :], target[:, :, 2, :, :]).mean()

        physics_loss = physics_loss_fn(output, target)

        num_time_steps = output.size(1)

        if time_step_DIV_losses_accum is None:
            time_step_DIV_losses_accum = np.zeros(num_time_steps, dtype=np.float32)

        if time_step_FF_losses_accum is None:
            time_step_FF_losses_accum = np.zeros(num_time_steps, dtype=np.float32)

        for i in range(num_time_steps):
            DIV_losses_step = single_physics_loss_DIV_fn(output[:, i, :, :, :], target[:, i, :, :, :])
            DIV_losses_step_scalar = DIV_losses_step.mean().item()

            time_step_DIV_losses_accum[i] += DIV_losses_step_scalar

        for i in range(num_time_steps):
            FF_losses_step = single_physics_loss_FF_fn(output[:, i, :, :, :], target[:, i, :, :, :])
            FF_losses_step_scalar = FF_losses_step.mean().item()

            time_step_FF_losses_accum[i] += FF_losses_step_scalar        

        total_loss = loss_Bx + loss_By + loss_Bz + physics_loss
        total_loss.backward()
        optimizer.step()

        epoch_loss_Bx += loss_Bx.item()
        epoch_loss_By += loss_By.item()
        epoch_loss_Bz += loss_Bz.item()
        epoch_physics_loss += physics_loss.item()

        if batch_idx % print_frequency == 0:
            train_loader_tqdm.set_postfix(
                loss_Bx=loss_Bx.item(),
                loss_By=loss_By.item(),
                loss_Bz=loss_Bz.item(),
                physics_loss=physics_loss.item()
            )

    avg_loss_Bx = epoch_loss_Bx / len(train_loader)
    avg_loss_By = epoch_loss_By / len(train_loader)
    avg_loss_Bz = epoch_loss_Bz / len(train_loader)
    avg_physics_loss = epoch_physics_loss / len(train_loader)

    train_losses_Bx.append(avg_loss_Bx)
    train_losses_By.append(avg_loss_By)
    train_losses_Bz.append(avg_loss_Bz)
    physics_losses_epoch.append(avg_physics_loss)

    all_E_pred = []
    all_E_true = []

    model.eval()
    epoch_r2_Bx = []
    epoch_r2_By = []
    epoch_r2_Bz = []
    test_loader_tqdm = tqdm(test_loader, total=len(test_loader), desc=f'Test Epoch {epoch+1}/{num_epochs}')
    with torch.no_grad():
        for sample in test_loader_tqdm:
            x = sample['input'].to(device0)
            lengths = sample['lengths'].to(device0)  
            target = sample['target'].cpu().numpy()
            output = newmodel(x, lengths).cpu().numpy()

            for true_sample, pred_sample in zip(target, output):
                r2_Bx = r2_score(true_sample[:, 0].flatten(), pred_sample[:, 0].flatten())
                r2_By = r2_score(true_sample[:, 1].flatten(), pred_sample[:, 1].flatten())
                r2_Bz = r2_score(true_sample[:, 2].flatten(), pred_sample[:, 2].flatten())

                epoch_r2_Bx.append(r2_Bx)
                epoch_r2_By.append(r2_By)
                epoch_r2_Bz.append(r2_Bz)


    avg_r2_Bx = np.mean(epoch_r2_Bx)
    avg_r2_By = np.mean(epoch_r2_By)
    avg_r2_Bz = np.mean(epoch_r2_Bz)

    test_losses_Bx.append(avg_r2_Bx)
    test_losses_By.append(avg_r2_By)
    test_losses_Bz.append(avg_r2_Bz)

    end_time = time.time()
    epoch_duration = end_time - start_time
    epoch_durations.append(epoch_duration)

    avg_train_loss_combined = (avg_loss_Bx + avg_loss_By + avg_loss_Bz + avg_physics_loss) / 3
    scheduler.step(avg_train_loss_combined)

    avg_epoch_duration = np.mean(epoch_durations)
    remaining_epochs = num_epochs - (epoch + 1)
    remaining_time = remaining_epochs * avg_epoch_duration
    hours, rem = divmod(remaining_time, 3600)
    minutes, seconds = divmod(rem, 60)

    print(f'Epoch: {epoch+1}/{num_epochs}, Duration: {epoch_duration:.2f}s, '
          f'Train Loss - Bx: {avg_loss_Bx:.4f}, By: {avg_loss_By:.4f}, Bz: {avg_loss_Bz:.4f}, Physics Loss: {avg_physics_loss:.4f}, '
          f'Test R2 - Bx: {avg_r2_Bx:.4f}, By: {avg_r2_By:.4f}, Bz: {avg_r2_Bz:.4f}')

    print(f'Estimated Remaining Time: {int(hours)}h {int(minutes)}m {int(seconds)}s')

    if (epoch + 1) >= save_checkpoint_start_epoch and (epoch + 1) % save_checkpoint_interval == 0:
        checkpoint_path = os.path.join(checkpoint_dir, f"checkpoint_epoch_{epoch+1}.pt")
        torch.save(newmodel.state_dict(), checkpoint_path)
        print(f"Checkpoint saved at epoch {epoch+1}: {checkpoint_path}")

        min_val = float('inf')
        max_val = float('-inf')

        for sample in test_loader:
            y_true_batch = sample['target'].numpy()
            batch_min = y_true_batch.min()
            batch_max = y_true_batch.max()
            min_val = min(min_val, batch_min)
            max_val = max(max_val, batch_max)

        data_range = max_val - min_val

        sample_r2_scores = {'bx': [], 'by': [], 'bz': []}
        sample_relative_errors = {'bx': [], 'by': [], 'bz': []}
        sample_mses = {'bx': [], 'by': [], 'bz': []}
        sample_maes = {'bx': [], 'by': [], 'bz': []}
        sample_psnr_values = {'bx': [], 'by': [], 'bz': []}
        sample_ssim_values = {'bx': [], 'by': [], 'bz': []}
        mean_std = {comp: {'mean': None, 'std': None} for comp in ['bx', 'by', 'bz']}
        true_values_by_component = {comp: [] for comp in ['bx', 'by', 'bz']}

        for sample in train_loader: 
            y_true_batch = sample['target'].cpu().numpy()
            for true_sample in y_true_batch:
                for i, comp in enumerate(['bx', 'by', 'bz']):
                    true_sample_component = true_sample[:, i]
                    true_sample_flat = true_sample_component.flatten()
                    true_values_by_component[comp].extend(true_sample_flat)

        for comp in ['bx', 'by', 'bz']:
            true_values = np.array(true_values_by_component[comp])
            mean_std[comp]['mean'] = np.mean(true_values)
            mean_std[comp]['std'] = np.std(true_values)

        for comp in mean_std:
            print(f"{comp} - Mean: {mean_std[comp]['mean']}, Std: {mean_std[comp]['std']}")

        model.eval()
        with torch.no_grad():
            for sample in test_loader:
                x = sample['input'].to(device0)
                lengths = sample['lengths'].to(device0)  
                y_true_batch = sample['target'].cpu().numpy()
                y_pred_batch = newmodel(x, lengths).cpu().numpy()

                for true_sample, pred_sample in zip(y_true_batch, y_pred_batch):
                    for i, comp in enumerate(['bx', 'by', 'bz']):
                        true_sample_component = true_sample[:, i]
                        pred_sample_component = pred_sample[:, i]

                        true_sample_flat = true_sample_component.flatten()
                        pred_sample_flat = pred_sample_component.flatten()

                        r2 = r2_score(true_sample_flat, pred_sample_flat)
                        sample_r2_scores[comp].append(r2)

                        mean = mean_std[comp]['mean']
                        std = mean_std[comp]['std']
                        true_sample_original = true_sample_flat * std + mean
                        pred_sample_original = pred_sample_flat * std + mean

                        absolute_error = np.linalg.norm(true_sample_original - pred_sample_original, 1)

                        relative_error = absolute_error / np.linalg.norm(true_sample_original, 1)
                        sample_relative_errors[comp].append(relative_error)

                        mse = mean_squared_error(true_sample_flat, pred_sample_flat)
                        sample_mses[comp].append(mse)

                        mae = mean_absolute_error(true_sample_flat, pred_sample_flat)
                        sample_maes[comp].append(mae)

                        psnr_value = psnr(true_sample_component, pred_sample_component, data_range=data_range)
                        sample_psnr_values[comp].append(psnr_value)

                        ssim_value = ssim(true_sample_component, pred_sample_component, data_range=data_range)
                        sample_ssim_values[comp].append(ssim_value)

        with open(results_file, mode='a', newline='') as file:
            writer = csv.writer(file)
            for comp in ['bx', 'by', 'bz']:
                writer.writerow([epoch+1, comp,
                                 np.mean(sample_r2_scores[comp]),
                                 np.mean(sample_relative_errors[comp]),
                                 np.mean(sample_mses[comp]),
                                 np.mean(sample_maes[comp]),
                                 np.mean(sample_psnr_values[comp]),
                                 np.mean(sample_ssim_values[comp])])

np.save(os.path.join(checkpoint_dir, 'train_losses_Bx.npy'), np.array(train_losses_Bx))
np.save(os.path.join(checkpoint_dir, 'train_losses_By.npy'), np.array(train_losses_By))
np.save(os.path.join(checkpoint_dir, 'train_losses_Bz.npy'), np.array(train_losses_Bz))
np.save(os.path.join(checkpoint_dir, 'test_losses_Bx.npy'), np.array(test_losses_Bx))
np.save(os.path.join(checkpoint_dir, 'test_losses_By.npy'), np.array(test_losses_By))
np.save(os.path.join(checkpoint_dir, 'test_losses_Bz.npy'), np.array(test_losses_Bz))

plt.figure(figsize=(10, 5))
plt.plot(train_losses_Bx, label='Training Loss Bx', color='red')
plt.plot(train_losses_By, label='Training Loss By', color='green')
plt.plot(train_losses_Bz, label='Training Loss Bz', color='blue')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Losses for Bx, By, Bz')
plt.legend()
plt.show()

plt.figure(figsize=(10, 5))
plt.plot(test_losses_Bx, label='Testing R2 Bx', color='red')
plt.plot(test_losses_By, label='Testing R2 By', color='green')
plt.plot(test_losses_Bz, label='Testing R2 Bz', color='blue')
plt.xlabel('Epoch')
plt.ylabel('R2 Score')
plt.title('Testing R2 Scores for Bx, By, Bz')
plt.legend()
plt.show()


In [ ]:
newmodel.load_state_dict(torch.load("result/NLFFF_Cube2Step/step2/checkpoint_epoch_140.pt"))
newmodel.eval() 

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch

def fig_to_rgb_array(fig):
    fig.canvas.draw()                               # 必须先绘制
    w, h = fig.canvas.get_width_height()
    buf = np.frombuffer(fig.canvas.buffer_rgba(), dtype=np.uint8)
    img = buf.reshape(h, w, 4)[..., :3]             # 去掉 Alpha 通道
    return img

output_dir = "result/NLFFF_Cube2Step/step2Test/visualizations"
os.makedirs(output_dir, exist_ok=True)

fps         = 3
frame_size  = (1024, 512)                           # (W, H) for VideoWriter
channel_ids = ["Bx", "By", "Bz"]

device0 = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

for sample_idx, sample in enumerate(test_dataset):

    input_data = sample["input"]                    # (T, 3, H, W)
    true_data  = sample["target"]                   # (T, 3, H, W)
    lengths    = sample["lengths"]

    with torch.no_grad():
        pred_batch = newmodel(
            input_data.unsqueeze(0).to(device0),    # (1, T, 3, H, W)
            lengths.to(device0)
        )
    pred_data = pred_batch.cpu().numpy()[0]         # (T, 3, H, W)

    time_length = pred_data.shape[0]

    for ch_idx, ch_name in enumerate(channel_ids):

        gt_path   = os.path.join(output_dir, f"gt_{ch_name}_sample_{sample_idx}.mp4")
        pred_path = os.path.join(output_dir, f"pred_{ch_name}_sample_{sample_idx}.mp4")

        gt_writer   = cv2.VideoWriter(gt_path,   cv2.VideoWriter_fourcc(*"mp4v"), fps, frame_size)
        pred_writer = cv2.VideoWriter(pred_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, frame_size)

        for t in range(time_length):

            true_frame = true_data[t, ch_idx]       # (H, W)
            pred_frame = pred_data[t, ch_idx]       # (H, W)

            vmin, vmax = true_frame.min(), true_frame.max()
            vmin = min(vmin, pred_frame.min())
            vmax = max(vmax, pred_frame.max())

            # ===== Ground Truth =====
            fig_gt, ax_gt = plt.subplots(figsize=(14, 7))
            plt.subplots_adjust(top=0.95, bottom=0.05, left=0.05, right=0.95)
            ax_gt.imshow(true_frame, cmap="RdBu", origin="lower", vmin=vmin, vmax=vmax)
            ax_gt.axis("off")

            gt_img = fig_to_rgb_array(fig_gt)
            gt_img = cv2.resize(gt_img, frame_size)
            gt_writer.write(cv2.cvtColor(gt_img, cv2.COLOR_RGB2BGR))
            plt.close(fig_gt)

            fig_pred, ax_pred = plt.subplots(figsize=(14, 7))
            plt.subplots_adjust(top=0.95, bottom=0.05, left=0.05, right=0.95)
            ax_pred.imshow(pred_frame, cmap="RdBu", origin="lower", vmin=vmin, vmax=vmax)
            ax_pred.axis("off")

            pred_img = fig_to_rgb_array(fig_pred)
            pred_img = cv2.resize(pred_img, frame_size)
            pred_writer.write(cv2.cvtColor(pred_img, cv2.COLOR_RGB2BGR))
            plt.close(fig_pred)

        gt_writer.release()
        pred_writer.release()
        print(f"[✓] sample {sample_idx} | {ch_name}: GT & Pred videos saved.")

print("✅ All videos have been generated.")


In [ ]:
import os
import numpy as np
import pandas as pd
import torch
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from skimage.metrics import structural_similarity as ssim

output_dir = "result/NLFFF_Cube2Step/step2Test/cube"
os.makedirs(output_dir, exist_ok=True)

npy_dir = os.path.join(output_dir, "npy_data")
os.makedirs(npy_dir, exist_ok=True)

csv_path = os.path.join(output_dir, "cube_metrics.csv")

def mean_relative_error(y_true, y_pred, zeroarr):

    return mean_absolute_error(y_true, y_pred) / mean_absolute_error(y_true,zeroarr)

def compute_psnr(y_true, y_pred):
    mse = np.mean((y_true - y_pred) ** 2)
    if mse == 0:
        return float("inf")
    data_range = np.max(y_true) - np.min(y_true)
    data_range = data_range if data_range > 0 else 1.0
    return 20 * np.log10(data_range / np.sqrt(mse))

def compute_ssim_cube(true_cube, pred_cube):
    ssim_vals = []
    for i in range(true_cube.shape[0]):
        data_range = true_cube[i].max() - true_cube[i].min()
        data_range = data_range if data_range > 0 else 1.0
        ssim_vals.append(
            ssim(true_cube[i], pred_cube[i], data_range=data_range)
        )
    return float(np.mean(ssim_vals))

rows = []

for sample_idx, sample in enumerate(test_dataset):

    input_data = sample["input"]          # (T, 3, H, W)  torch.Tensor
    true_data  = sample["target"]         # (T, 3, H, W)  torch.Tensor
    lengths    = sample["lengths"]        # (1,)          torch.Tensor

    print(true_data.shape)

    # 2) 推理
    with torch.no_grad():
        pred_batch = newmodel(
            input_data.unsqueeze(0).to(device0),  # (1, T, 3, H, W)
            lengths.to(device0)
        )
    pred_data = pred_batch.cpu().numpy()[0]       # (T, 3, H, W)
    true_data_np = true_data.cpu().numpy()        # (T, 3, H, W)

    for channel_idx, channel_name in enumerate(["Bx", "By", "Bz"]):
        pred_cube  = pred_data[:, channel_idx, :, :]    # (T, H, W)
        true_cube  = true_data_np[:, channel_idx, :, :]

        np.save(
            os.path.join(npy_dir, f"sample_{sample_idx}_{channel_name}_pred.npy"),
            pred_cube
        )

        y_true = true_cube.flatten()
        y_pred = pred_cube.flatten()
        

        r2_val   = r2_score(y_true, y_pred)
        mse_val  = mean_squared_error(y_true, y_pred)
        mae_val  = mean_absolute_error(y_true, y_pred)
        zeroarr = np.zeros(y_true.shape)
        re_val   = mean_relative_error(y_true, y_pred, zeroarr)
        psnr_val = compute_psnr(true_cube, pred_cube)
        ssim_val = compute_ssim_cube(true_cube, pred_cube)

        rows.append({
            "sample":  sample_idx,
            "channel": channel_name,
            "R2":      r2_val,
            "MSE":     mse_val,
            "MAE":     mae_val,
            "RE":      re_val,
            "PSNR":    psnr_val,
            "SSIM":    ssim_val
        })

        print(f"[Sample {sample_idx} - {channel_name}] "
              f"R2={r2_val:.4f}, MSE={mse_val:.3e}, MAE={mae_val:.3e}, "
              f"RE={re_val:.4f}, PSNR={psnr_val:.2f}, SSIM={ssim_val:.4f}")

# 4) 保存 CSV
metrics_df = pd.DataFrame(rows)
metrics_df.to_csv(csv_path, index=False)

